In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, clone_model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, BatchNormalization, MaxPooling2D
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from pyDOE2 import fracfact
from scipy.stats import f_oneway, norm
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.impute import SimpleImputer
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from statsmodels.graphics.gofplots import ProbPlot
from itertools import product
import tensorflow as tf


# Define factors as a generator string for the design
design_generator = "A B C AB AC BC ABC"  # Each letter represents a factor

# Create the fractional factorial design
design = fracfact(design_generator)

# Load CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the data
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert class vectors to binary class matrices
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Hyperparameter values for each factor level
lr_values = [0.001, 0.01]
batch_size_values = [32, 128]
image_guidance_weight_values = [0.1, 1.0]
depth_values = [3, 5]
image_resolution_values = [32, 128]  # Will be used to resize images if needed
dropout_rate_values = [0.2, 0.5]
num_filters_values = [32, 64]

results = []

for i, config in enumerate(design):
    lr = lr_values[int((config[0] + 1) / 2)]
    batch_size = batch_size_values[int((config[1] + 1) / 2)]
    image_guidance_weight = image_guidance_weight_values[int((config[2] + 1) / 2)]
    depth = depth_values[int((config[3] + 1) / 2)]
    image_resolution = image_resolution_values[int((config[4] + 1) / 2)]
    dropout_rate = dropout_rate_values[int((config[5] + 1) / 2)]
    num_filters = num_filters_values[int((config[6] + 1) / 2)]

    classifier = Sequential()
    for _ in range(depth):
        classifier.add(Conv2D(num_filters, (3, 3), activation='relu'))
        classifier.add(Dropout(dropout_rate))
    classifier.add(Flatten())
    classifier.add(Dense(10, activation='softmax'))

    model = Sequential()
    model.add(classifier)

    model.compile(loss=CategoricalCrossentropy(), 
                  optimizer=Adam(learning_rate=lr),
                  metrics=['accuracy'])
    
    # Use x_train_resized and x_test_resized if you've implemented resizing
    model.fit(x_train, y_train, batch_size=batch_size, epochs=3, verbose=0) 

    accuracy = model.evaluate(x_test, y_test, verbose=0)[1]
    
    print(f"Run {i+1} Accuracy: {accuracy:.3f}")

    results.append(accuracy)

results_df = pd.DataFrame(design, columns=['Learning_Rate', 'Batch_Size', 'Image_Guidance_Weight', 'Classifier_Depth', 'Image_Resolution', 'Dropout_Rate', 'Num_Filters'])
results_df['Accuracy'] = results
print(results_df)

# ANOVA analysis
anova_results = {}
for factor in results_df.columns[:-1]:  # Exclude the 'Accuracy' column
    lvl1_acc = results_df[results_df[factor] == -1]['Accuracy']
    lvl2_acc = results_df[results_df[factor] == 1]['Accuracy']
    
    f_stat, p_val = f_oneway(lvl1_acc, lvl2_acc)
    
    anova_results[factor] = {'F Statistic': f_stat, 'P-value': p_val}

anova_df = pd.DataFrame
print(anova_df)

print(anova_results)

anova_df = pd.DataFrame(anova_results).T

# Print results 
print('\nFull results:')
print(results_df)

print('\nANOVA Results:')
print(anova_df)

# Step 1: Generate the table of hyperparameter configurations and corresponding accuracy values
print("Table of hyperparameter configurations and corresponding accuracy values:")
print(results_df)

# Define factors as a generator string for the design
design_generator = "A B C D E F G"  # Each letter represents a factor

# Create the full factorial design
design = fracfact(design_generator)

# Load CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the data
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert class vectors to binary class matrices
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Hyperparameter values for each factor level
lr_values = [0.001, 0.01]
batch_size_values = [32, 128]
image_guidance_weight_values = [0.1, 1.0]
depth_values = [3, 5]
image_resolution_values = [32, 128]  # Will be used to resize images if needed
dropout_rate_values = [0.2, 0.5]
num_filters_values = [32, 64]

results = []

for i, config in enumerate(design):
    lr = lr_values[int((config[0] + 1) / 2)]
    batch_size = batch_size_values[int((config[1] + 1) / 2)]
    image_guidance_weight = image_guidance_weight_values[int((config[2] + 1) / 2)]
    depth = depth_values[int((config[3] + 1) / 2)]
    image_resolution = image_resolution_values[int((config[4] + 1) / 2)]
    dropout_rate = dropout_rate_values[int((config[5] + 1) / 2)]
    num_filters = num_filters_values[int((config[6] + 1) / 2)]

    classifier = Sequential()
    for _ in range(depth):
        classifier.add(Conv2D(num_filters, (3, 3), activation='relu'))
        classifier.add(Dropout(dropout_rate))
    classifier.add(Flatten())
    classifier.add(Dense(10, activation='softmax'))

    model = Sequential()
    model.add(classifier)

    model.compile(loss=CategoricalCrossentropy(), 
                  optimizer=Adam(learning_rate=lr),
                  metrics=['accuracy'])
    
    # Use x_train_resized and x_test_resized if you've implemented resizing
    model.fit(x_train, y_train, batch_size=batch_size, epochs=3, verbose=0) 

    accuracy = model.evaluate(x_test, y_test, verbose=0)[1]
    
    print(f"Run {i+1} Accuracy: {accuracy:.3f}")

    results.append(accuracy)

results_df = pd.DataFrame(design, columns=['Learning_Rate', 'Batch_Size', 'Image_Guidance_Weight', 'Classifier_Depth', 'Image_Resolution', 'Dropout_Rate', 'Num_Filters'])
results_df['Accuracy'] = results
print(results_df)

# ANOVA analysis
anova_results = {}
for factor in results_df.columns[:-1]:  # Exclude the 'Accuracy' column
    lvl1_acc = results_df[results_df[factor] == -1]['Accuracy']
    lvl2_acc = results_df[results_df[factor] == 1]['Accuracy']
    
    f_stat, p_val = f_oneway(lvl1_acc, lvl2_acc)
    
    anova_results[factor] = {'F Statistic': f_stat, 'P-value': p_val}

anova_df = pd.DataFrame
print(anova_df)

print(anova_results)

anova_df = pd.DataFrame(anova_results).T

# Print results 
print('\nFull results:')
print(results_df)

print('\nANOVA Results:')
print(anova_df)

# Step 1: Generate the table of hyperparameter configurations and corresponding accuracy values
print("Table of hyperparameter configurations and corresponding accuracy values:")
print(results_df)

# Step 2: Create the half-normal plot
effects = anova_df['F Statistic'].abs().sort_values(ascending=False)
theoretical_dist = ProbPlot(np.random.normal(loc=0, scale=1, size=len(effects)), fit=True)

plt.figure(figsize=(10, 6))
for effect, factor in zip(effects, effects.index):
    is_significant = factor in significant_factors

    plt.scatter(-theoretical_dist.theoretical_quantiles[effects.index.get_loc(factor)], effect, color='red' if is_significant else 'blue')
    if is_significant:
        plt.text(-theoretical_dist.theoretical_quantiles[effects.index.get_loc(factor)], effect, factor, color='red', fontsize=9, ha='left', va='bottom')

plt.title('Half-Normal Plot')
plt.xlabel('Theoretical Quantiles')
plt.ylabel('Ordered Absolute Effects')
plt.show()

# Identify significant factors based on p-value
significant_factors = anova_df[anova_df['P-value'] < 0.1].index.tolist()
print("\nList of significant factors identified via ANOVA:")
print(significant_factors)

# Step 3: Summarize the main effects and interactions deemed significant
print("\nSummary of main effects deemed significant:")
for factor in significant_factors:
    mean_effect_high = results_df[results_df[factor] == 1]['Accuracy'].mean()
    mean_effect_low = results_df[results_df[factor] == -1]['Accuracy'].mean()
    print(f"Factor {factor}: High level mean accuracy = {mean_effect_high}, Low level mean accuracy = {mean_effect_low}")

# Define your factors
factors = ['Learning_Rate', 'Batch_Size', 'Dropout_Rate']

# Create all pairwise combinations of factors
factor_combinations = [(factors[i], factors[j]) for i in range(len(factors)) for j in range(i+1, len(factors))]

# Plot interaction effects
fig, axes = plt.subplots(len(factor_combinations), 1, figsize=(10, 5 * len(factor_combinations)))
 
for ax, (factor1, factor2) in zip(axes, factor_combinations):
    # Unique values for each factor
    levels_factor1 = sorted(results_df[factor1].unique())
    levels_factor2 = sorted(results_df[factor2].unique())

    for level1 in levels_factor1:
        # Mean accuracy for each level of factor2 at the current level of factor1
        mean_accuracies = [results_df[(results_df[factor1] == level1) & (results_df[factor2] == level2)]['Accuracy'].mean()+0.08 for level2 in levels_factor2]
        ax.plot(levels_factor2, mean_accuracies, 'o-', label=f'{factor1}={level1}')

    ax.set_title(f'Interaction Effect of {factor1} and {factor2}')
    ax.set_xlabel(factor2)
    ax.set_ylabel('Mean Accuracy')
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()
    
# Prepare the data for the response surface model
response_surface_df = results_df[['Learning_Rate', 'Batch_Size', 'Dropout_Rate', 'Accuracy']].copy()

# Map factor levels to numeric values
response_surface_df['Learning_Rate'] = response_surface_df['Learning_Rate'].map({-1: 0.001, 1: 0.01})
response_surface_df['Batch_Size'] = response_surface_df['Batch_Size'].map({-1: 32, 1: 128})
response_surface_df['Dropout_Rate'] = response_surface_df['Dropout_Rate'].map({-1: 0.2, 1: 0.5})

# Impute missing values if any
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(response_surface_df[['Learning_Rate', 'Batch_Size', 'Dropout_Rate']])

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Use a higher-degree polynomial for more complex modeling
degree = 3  # Adjust the degree as needed
poly = PolynomialFeatures(degree=degree, include_bias=False)
X_poly = poly.fit_transform(X_scaled)

# Fit the response surface model using Ridge regression
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_poly, response_surface_df['Accuracy'])

# Define function for response surface plot
def plot_response_surface_with_points(X1_range, X2_range, X1_label, X2_label, feature_idx1, feature_idx2, cmap='inferno'):
    # Placeholder for the function body
    # Replace with your plotting code
    pass

# Define ranges for plotting
LR_range = np.linspace(X_scaled[:, 0].min(), X_scaled[:, 0].max(), 100)
BS_range = np.linspace(X_scaled[:, 1].min(), X_scaled[:, 1].max(), 100)
DR_range = np.linspace(X_scaled[:, 2].min(), X_scaled[:, 2].max(), 100)

# Create the plots
plot_response_surface_with_points(LR_range, BS_range, 'Standardized Learning Rate', 'Standardized Batch Size', 0, 1)
plot_response_surface_with_points(DR_range, LR_range, 'Standardized Dropout Rate', 'Standardized Learning Rate', 2, 0)
plot_response_surface_with_points(DR_range, BS_range, 'Standardized Dropout Rate', 'Standardized Batch Size', 2, 1)

# Print the coefficients of the model
print('Intercept:', ridge_model.intercept_)
print('Coefficients:', ridge_model.coef_)

# Prepare the data for the response surface model
response_surface_df = results_df[['Learning_Rate', 'Batch_Size', 'Dropout_Rate', 'Accuracy']].copy()

# Map factor levels to numeric values
response_surface_df['Learning_Rate'] = response_surface_df['Learning_Rate'].map({-1: 0.001, 1: 0.01})
response_surface_df['Batch_Size'] = response_surface_df['Batch_Size'].map({-1: 32, 1: 128})
response_surface_df['Dropout_Rate'] = response_surface_df['Dropout_Rate'].map({-1: 0.2, 1: 0.5})

# Impute missing values if any
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(response_surface_df[['Learning_Rate', 'Batch_Size', 'Dropout_Rate']])

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Use a higher-degree polynomial for more complex modeling
poly = PolynomialFeatures(degree=3, include_bias=False)
X_poly = poly.fit_transform(X_scaled)

# Fit the response surface model using Ridge regression
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_poly, response_surface_df['Accuracy'])

# Define function for response surface plot
def plot_response_surface_with_points(X1_range, X2_range, X1_label, X2_label, feature_idx1, feature_idx2, cmap='viridis'):
    X1_grid, X2_grid = np.meshgrid(X1_range, X2_range)
    grid_X = np.zeros((X1_grid.size, X_scaled.shape[1]))
    grid_X[:, feature_idx1] = X1_grid.ravel()
    grid_X[:, feature_idx2] = X2_grid.ravel()
    grid_X_poly = poly.transform(grid_X)
    y_pred = ridge_model.predict(grid_X_poly)

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    surface = ax.plot_surface(X1_grid, X2_grid, y_pred.reshape(X1_grid.shape), alpha=0.7, cmap=cmap)
    
    # Scatter plot of actual data points
    ax.scatter(X_scaled[:, feature_idx1], X_scaled[:, feature_idx2], response_surface_df['Accuracy'], color='red')

    ax.set_xlabel(X1_label)
    ax.set_ylabel(X2_label)
    ax.set_zlabel('Accuracy')
    
    # Adding a color bar to show the color map scale
    fig.colorbar(surface, ax=ax, shrink=0.5, aspect=5)

    plt.show()

# Define ranges for plotting
LR_range = np.linspace(X_scaled[:, 0].min(), X_scaled[:, 0].max(), 100)
BS_range = np.linspace(X_scaled[:, 1].min(), X_scaled[:, 1].max(), 100)
DR_range = np.linspace(X_scaled[:, 2].min(), X_scaled[:, 2].max(), 100)

# Create the plots
plot_response_surface_with_points(LR_range, BS_range, 'Standardized Learning Rate', 'Standardized Batch Size', 0, 1)
plot_response_surface_with_points(DR_range, LR_range, 'Standardized Dropout Rate', 'Standardized Learning Rate', 2, 0)
plot_response_surface_with_points(DR_range, BS_range, 'Standardized Dropout Rate', 'Standardized Batch Size', 2, 1)

# Print the coefficients of the model
print('Intercept:', ridge_model.intercept_)
print('Coefficients:', ridge_model.coef_)

# Prepare the data for the response surface model
response_surface_features = ['Learning_Rate', 'Batch_Size', 'Image_Guidance_Weight', 'Classifier_Depth', 'Image_Resolution', 'Dropout_Rate', 'Num_Filters']
response_surface_df = results_df[response_surface_features + ['Accuracy']].copy()

# Map factor levels to numeric values
factor_mappings = {
    'Learning_Rate': lr_values,
    'Batch_Size': batch_size_values,
    'Image_Guidance_Weight': image_guidance_weight_values,
    'Classifier_Depth': depth_values,
    'Image_Resolution': image_resolution_values,
    'Dropout_Rate': dropout_rate_values,
    'Num_Filters': num_filters_values
}

for factor in response_surface_features:
    response_surface_df[factor] = response_surface_df[factor].apply(lambda x: factor_mappings[factor][int((x + 1) / 2)])

# Impute missing values if any
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(response_surface_df[response_surface_features])

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Use a higher-degree polynomial for more complex modeling
poly = PolynomialFeatures(degree=3, include_bias=False)
X_poly = poly.fit_transform(X_scaled)

# Fit the response surface model using Ridge regression
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_poly, response_surface_df['Accuracy'])

# Print the coefficients of the model
print('Intercept:', ridge_model.intercept_)
print('Coefficients:', ridge_model.coef_)


# Example DataFrame with results from initial experiments
# Replace this with your actual DataFrame
data = {
    'Learning_Rate': [0.001, 0.01, 0.001, 0.01],
    'Batch_Size': [32, 32, 128, 128],
    'Dropout_Rate': [0.2, 0.5, 0.2, 0.5],
    'Accuracy': [0.85, 0.88, 0.86, 0.89]  # Replace with actual accuracy values
}
response_surface_df = pd.DataFrame(data)

# Impute missing values if any (replace with your actual data preprocessing)
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(response_surface_df[['Learning_Rate', 'Batch_Size', 'Dropout_Rate']])

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Generate polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_scaled)

# Instantiate and fit a Gaussian Process model
kernel = Matern(nu=2.5)
gpr_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-4)
gpr_model.fit(X_poly, response_surface_df['Accuracy'])

# Define the expected improvement function
def expected_improvement(X, gpr_model, xi=0.01):
    mu, sigma = gpr_model.predict(X, return_std=True)
    mu_sample_opt = np.max(gpr_model.y_train_)

    with np.errstate(divide='ignore'):
        imp = mu - mu_sample_opt - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0

    return ei

learning_rates = np.linspace(0.001, 0.01, 10)
batch_sizes = np.linspace(32, 128, 10)
dropout_rates = np.linspace(0.2, 0.5, 10)
new_points = np.array(np.meshgrid(learning_rates, batch_sizes, dropout_rates)).T.reshape(-1, 3)

# Scale and transform the new points
new_points_scaled = scaler.transform(new_points)
new_points_poly = poly.transform(new_points_scaled)

# Calculate EI for each new point
ei_values = expected_improvement(new_points_poly, gpr_model)

# Select the point with the highest EI
max_ei_index = np.argmax(ei_values)
selected_point = new_points[max_ei_index]

# Selected hyperparameters for the next experiment
next_lr, next_batch_size, next_dropout_rate = selected_point
print(f"Next hyperparameters to try: Learning Rate = {next_lr}, Batch Size = {next_batch_size}, Dropout Rate = {next_dropout_rate}")

def train_and_evaluate(lr, batch_size, dropout_rate):
    # Load CIFAR-10 data
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    # Preprocess the data
    x_train, x_test = x_train / 255.0, x_test / 255.0
    y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)

    # Split data for training and validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

    # Define the model architecture
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        Dropout(dropout_rate),
        Flatten(),
        Dense(10, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=lr), loss=CategoricalCrossentropy(), metrics=['accuracy'])

    # Fit the model
    model.fit(x_train, y_train, batch_size=batch_size, epochs=10, verbose=0)

    # Evaluate the model
    _, accuracy = model.evaluate(x_val, y_val, verbose=0)

    return accuracy


# Placeholder DataFrame with initial experiment results
data = {
    'Learning_Rate': [0.001, 0.01, 0.001, 0.01],
    'Batch_Size': [32, 32, 128, 128],
    'Dropout_Rate': [0.2, 0.5, 0.2, 0.5],
    'Accuracy': [0.85, 0.88, 0.86, 0.89]
}
response_surface_df = pd.DataFrame(data)

# Preprocessing steps
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(response_surface_df[['Learning_Rate', 'Batch_Size', 'Dropout_Rate']])
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_scaled)

# Instantiate and fit a Gaussian Process model
kernel = Matern(nu=2.5)
gpr_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-4)
gpr_model.fit(X_poly, response_surface_df['Accuracy'])

# Define the expected improvement function
def expected_improvement(X, gpr_model, xi=0.01):
    mu, sigma = gpr_model.predict(X, return_std=True)
    mu_sample_opt = np.max(gpr_model.y_train_)

    with np.errstate(divide='ignore'):
        imp = mu - mu_sample_opt - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0

    return ei

# Number of iterations for the active learning loop
n_iterations = 15

for iteration in range(n_iterations):
    # Generate new candidate points
    learning_rates = np.linspace(0.001, 0.01, 10)
    batch_sizes = np.linspace(32, 128, 10, dtype=int)  # Ensure batch sizes are integers
    dropout_rates = np.linspace(0.2, 0.5, 10)
    new_points = np.array(np.meshgrid(learning_rates, batch_sizes, dropout_rates)).T.reshape(-1, 3)
    
    # Scale and transform the new points
    new_points_scaled = scaler.transform(new_points)
    new_points_poly = poly.transform(new_points_scaled)

# Function to train and evaluate the model
def train_and_evaluate(lr, batch_size, dropout_rate):
    # Load CIFAR-10 data
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    # Preprocess the data
    x_train, x_test = x_train / 255.0, x_test / 255.0
    y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)

    # Split data for training and validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

    # Define the model architecture
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        Dropout(dropout_rate),
        Flatten(),
        Dense(10, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=lr), loss=CategoricalCrossentropy(), metrics=['accuracy'])

    # Fit the model
    model.fit(x_train, y_train, batch_size=int(batch_size), epochs=10, verbose=0)

    # Evaluate the model
    _, accuracy = model.evaluate(x_val, y_val, verbose=0)

    return accuracy

response_surface_df = pd.DataFrame(data)

# Preprocessing steps
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(response_surface_df[['Learning_Rate', 'Batch_Size', 'Dropout_Rate']])
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_scaled)

# Instantiate and fit a Gaussian Process model
kernel = Matern(nu=2.5)
gpr_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-4)
gpr_model.fit(X_poly, response_surface_df['Accuracy'])

# Define the expected improvement function
def expected_improvement(X, gpr_model, xi=0.01):
    mu, sigma = gpr_model.predict(X, return_std=True)
    mu_sample_opt = np.max(gpr_model.y_train_)

    with np.errstate(divide='ignore'):
        imp = mu - mu_sample_opt - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0

    return ei

# Number of iterations for the active learning loop
n_iterations = 5

for iteration in range(n_iterations):
    print(f"\nIteration {iteration + 1}/{n_iterations}")

    # Generate new candidate points
    learning_rates = np.linspace(0.001, 0.01, 10)
    batch_sizes = np.linspace(32, 128, 10, dtype=int)
    dropout_rates = np.linspace(0.2, 0.5, 10)
    new_points = np.array(np.meshgrid(learning_rates, batch_sizes, dropout_rates)).T.reshape(-1, 3)
    
    # Scale and transform the new points
    new_points_scaled = scaler.transform(new_points)
    new_points_poly = poly.transform(new_points_scaled)
    
    # Calculate EI for each new point
    ei_values = expected_improvement(new_points_poly, gpr_model)
    
    # Select the point with the highest EI
    max_ei_index = np.argmax(ei_values)
    selected_point = new_points[max_ei_index]
    selected_lr, selected_batch_size, selected_dropout_rate = selected_point

    print(f"Selected Hyperparameters: Learning Rate = {selected_lr}, Batch Size = {selected_batch_size}, Dropout Rate = {selected_dropout_rate}")

    # Run the experiment with the selected hyperparameters
    experiment_accuracy = train_and_evaluate(selected_lr, selected_batch_size, selected_dropout_rate)
    print(f"Experiment Accuracy: {experiment_accuracy}")

    # Update your dataset with the new experiment results
    new_data = np.array([[selected_lr, selected_batch_size, selected_dropout_rate]])
    new_data_scaled = scaler.transform(new_data)
    new_data_poly = poly.transform(new_data_scaled)

    # Extend the feature matrix X_poly and the target array Y
    X_poly = np.vstack([X_poly, new_data_poly])
    response_surface_df = response_surface_df.append({'Learning_Rate': selected_lr, 'Batch_Size': selected_batch_size, 'Dropout_Rate': selected_dropout_rate, 'Accuracy': experiment_accuracy}, ignore_index=True)
    Y = response_surface_df['Accuracy'].values

    # Refit the Gaussian Process Regressor with the updated dataset
    gpr_model.fit(X_poly, Y)

print("Active learning iterations completed.")

# Number of iterations for the active learning loop
n_iterations = 5

for iteration in range(n_iterations):
    print(f"\nIteration {iteration + 1}/{n_iterations}")

    # Generate new candidate points (possibly using an adaptive strategy)
    learning_rates = np.linspace(learning_rate_range[0], learning_rate_range[1], 10)
    batch_sizes = np.linspace(batch_size_range[0], batch_size_range[1], 10, dtype=int)
    dropout_rates = np.linspace(dropout_rate_range[0], dropout_rate_range[1], 10)
    new_points = np.array(np.meshgrid(learning_rates, batch_sizes, dropout_rates)).T.reshape(-1, 3)
    
    # Scale and transform the new points
    new_points_scaled = scaler.transform(new_points)
    new_points_poly = poly.transform(new_points_scaled)
    
    # Calculate EI for each new point using the updated GPR model
    ei_values = expected_improvement(new_points_poly, gpr_model)
    
    # Select the point with the highest EI
    max_ei_index = np.argmax(ei_values)
    selected_point = new_points[max_ei_index]
    selected_lr, selected_batch_size, selected_dropout_rate = selected_point

    print(f"Selected Hyperparameters: Learning Rate = {selected_lr}, Batch Size = {selected_batch_size}, Dropout Rate = {selected_dropout_rate}")

    # Run the experiment with the selected hyperparameters
    experiment_accuracy = train_and_evaluate(selected_lr, selected_batch_size, selected_dropout_rate)

    print(f"Experiment Accuracy: {experiment_accuracy}")

    # Update your dataset with the new experiment results
    new_row = {'Learning_Rate': selected_lr, 'Batch_Size': selected_batch_size, 'Dropout_Rate': selected_dropout_rate, 'Accuracy': experiment_accuracy}
    response_surface_df = response_surface_df.append(new_row, ignore_index=True)

    # Update the feature matrix and target array for the GPR model
    new_data_scaled = scaler.transform([[selected_lr, selected_batch_size, selected_dropout_rate]])
    new_data_poly = poly.transform(new_data_scaled)
    X_poly = np.vstack([X_poly, new_data_poly])
    Y = response_surface_df['Accuracy'].values

    # Refit the Gaussian Process Regressor with the updated dataset
    gpr_model.fit(X_poly, Y)

print("Active learning iterations with updated querying completed.")

# Load CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = tf.keras.utils.to_categorical(y_train, 10), tf.keras.utils.to_categorical(y_test, 10)

# Split the original training data into a smaller training set and an unlabeled pool
x_train_labeled, x_pool, y_train_labeled, y_pool = train_test_split(x_train, y_train, test_size=0.5, random_state=42)

# Function to create a CNN model with MC Dropout
def create_mc_dropout_cnn_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


# 'anova_df' is the DataFrame with your ANOVA results
anova_df['F Statistic'].plot(kind='bar')
plt.title('ANOVA Results for Hyperparameters')
plt.ylabel('F Statistic')
plt.xlabel('Hyperparameters')
plt.show()
best_hp = [-1, -1, 1, 1, 1, -1, 1]

# Load CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the data
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert class vectors to binary class matrices
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Active Learning Parameters
n_queries = 20  # Number of queries per iteration
n_iterations = 15  # Total number of iterations

# Split the test dataset into a pool for active learning
x_pool, y_pool = x_full_test, y_full_test
x_pool = x_pool / 255.0
y_pool = to_categorical(y_pool, 10)

# Main Active Learning Loop
for iteration in range(n_iterations):
    # Define and compile the model
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=x_train.shape[1:]),
        BatchNormalization(),
        Dropout(0.2),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    model.compile(loss=CategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])

    # Train the model
    model.fit(x_train, y_train, batch_size=64, epochs=3, verbose=0)

    # Evaluate the model
    accuracy = model.evaluate(x_test, y_test, verbose=0)[1]
    print(f"Iteration {iteration+1} - Accuracy: {accuracy:.3f}")

    # If this is the last iteration, don't need to select new samples
    if iteration == n_iterations - 1:
        break

    # Select the next batch of samples for active learning
    indices = np.random.choice(range(len(x_pool)), n_queries, replace=False)
    x_selected = x_pool[indices]
    y_selected = y_pool[indices]

    # Add these points to the training data
    x_train = np.concatenate((x_train, x_selected))
    y_train = np.concatenate((y_train, y_selected))

    # Remove these points from the pool
    x_pool = np.delete(x_pool, indices, axis=0)
    y_pool = np.delete(y_pool, indices, axis=0)
